In [5]:
import sys
sys.path.append("..")

In [60]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import ast

torch.manual_seed(1)
import pandas as pd
import numpy as np
import gensim
import torchtext
from torchtext import data
from utils.db_client import DBClient

In [57]:
db = DBClient(secrets_path='../configs/db_secrets.json')

I'm trying
Connected to political tweets DB


In [69]:
examples = db.read("select tweet_text_clean, 1 from staging.master limit 10")
#example = examples[0]
examples = [(ast.literal_eval(example[0]), example[1]) for example in examples]

In [70]:
examples

[(['side',
   'need',
   'learn',
   'firearm',
   'current',
   'process',
   'weve',
   'get',
   'keep',
   'firearm',
   'secure',
   'minor',
   'people',
   'restrain',
   'order',
   'shouldnt',
   'weapon',
   'need',
   'improve',
   'current',
   'background',
   'check',
   'system'],
  1),
 (['meeting',
   'north',
   'korea',
   'cancel',
   'northrup',
   'grummans',
   'stock',
   'soar',
   'love',
   'work',
   'corporation',
   '#american',
   'people',
   'receive',
   '36000',
   'ng',
   'buy',
   'let',
   '#change',
   '#congress',
   'put',
   '#people',
   '#politics',
   '#va08',
   '#gop'],
  1),
 (['west',
   'coast',
   'attempt',
   'literally',
   'limit',
   'first',
   'amendment',
   'punish',
   'business',
   'create',
   'job',
   'idolized',
   'worship',
   'pornstar',
   'reason',
   'try',
   'make',
   'trump',
   'look',
   'bad',
   'thank',
   'god',
   'im',
   'safe',
   'side',
   'country'],
  1),
 (['say',
   'everyone',
   'play',
   '

In [49]:
# load word2vec model
model = gensim.models.KeyedVectors.load_word2vec_format('../model/GoogleNews-vectors-negative300.bin', binary=True) 

In [ ]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

In [73]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

word_to_ix = {}
for tweet, tag in examples:
    for word in tweet:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {0: 0,  4: 1}

{'side': 0, 'need': 1, 'learn': 2, 'firearm': 3, 'current': 4, 'process': 5, 'weve': 6, 'get': 7, 'keep': 8, 'secure': 9, 'minor': 10, 'people': 11, 'restrain': 12, 'order': 13, 'shouldnt': 14, 'weapon': 15, 'improve': 16, 'background': 17, 'check': 18, 'system': 19, 'meeting': 20, 'north': 21, 'korea': 22, 'cancel': 23, 'northrup': 24, 'grummans': 25, 'stock': 26, 'soar': 27, 'love': 28, 'work': 29, 'corporation': 30, '#american': 31, 'receive': 32, '36000': 33, 'ng': 34, 'buy': 35, 'let': 36, '#change': 37, '#congress': 38, 'put': 39, '#people': 40, '#politics': 41, '#va08': 42, '#gop': 43, 'west': 44, 'coast': 45, 'attempt': 46, 'literally': 47, 'limit': 48, 'first': 49, 'amendment': 50, 'punish': 51, 'business': 52, 'create': 53, 'job': 54, 'idolized': 55, 'worship': 56, 'pornstar': 57, 'reason': 58, 'try': 59, 'make': 60, 'trump': 61, 'look': 62, 'bad': 63, 'thank': 64, 'god': 65, 'im': 66, 'safe': 67, 'country': 68, 'say': 69, 'everyone': 70, 'play': 71, 'game': 72, 'regard': 73,

In [79]:
class LSTMSentiment(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMSentiment, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.weights = torch.FloatTensor(model.wv.vectors)
        self.word_embeddings = nn.Embedding.from_pretrained(self.weights)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [88]:
lstm_model = LSTMSentiment(300, 3, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr=0.1)

training_data = examples

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = lstm_model(inputs)
    print(tag_scores)

for epoch in range(2):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        lstm_model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = lstm_model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.3146, -0.3128],
        [-1.3141, -0.3130],
        [-1.3186, -0.3113],
        [-1.2697, -0.3298],
        [-1.3589, -0.2970],
        [-1.3421, -0.3029],
        [-1.3518, -0.2995],
        [-1.3694, -0.2934],
        [-1.3176, -0.3117],
        [-1.2744, -0.3279],
        [-1.3539, -0.2987],
        [-1.4093, -0.2801],
        [-1.4203, -0.2766],
        [-1.3962, -0.2844],
        [-1.3570, -0.2976],
        [-1.3935, -0.2853],
        [-1.3532, -0.2990],
        [-1.4056, -0.2813],
        [-1.3572, -0.2976],
        [-1.4032, -0.2821],
        [-1.2545, -0.3358],
        [-1.2358, -0.3433],
        [-1.3295, -0.3073]])


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


TypeError: 'int' object is not iterable

In [81]:
lstm_model

LSTMSentiment(
  (word_embeddings): Embedding(3000000, 300)
  (lstm): LSTM(3, 3)
  (hidden2tag): Linear(in_features=3, out_features=2, bias=True)
)